In [1]:
from __future__ import print_function, division

from ourwrnet import *
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.datasets import cifar10
from keras.callbacks import LearningRateScheduler
from keras.models import model_from_json
from keras.models import load_model
from keras.losses import KLDivergence
import numpy as np
import keras
import math
import time
import matplotlib.pyplot as plt
import sys
import tensorflow as tf
import numpy as np

from keras.models import Model
from keras.layers import Input, Add, Activation, Dropout, Flatten, Dense
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K

#import wide_residual_network as wrn
from keras.datasets import cifar10
import keras.callbacks as callbacks
import keras.utils.np_utils as kutils
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Concatenate
from keras import backend as K
from keras.datasets import cifar10
from keras.callbacks import LearningRateScheduler
import numpy as np
from keras import optimizers

import torch.nn.functional as F
from cifar10utils import getCIFAR10, getCIFAR10InputShape

'''
Function that returns the network to train
'''
def getNetwork(input_shape):
    model, m1, m2, m3=create_wide_residual_network(input_shape, 10, N=2, k=2, dropout=0.)
    return model,m1,m2,m3;

'''
Function that saves a model on the disk
'''
def saveModel(model,filename):
    model_json = model.to_json()
    with open(filename + '.json', 'w') as json_file:
        json_file.write(model_json)
    model.save_weights(filename + '.h5') 

'''
Learning rate scheduler
'''
def lr_schedule(epoch):
    total_epochs = 130
    lrate = 0.1
    if epoch > total_epochs*0.3:
        lrate = 0.02
    if epoch > total_epochs*0.6:
        lrate = 0.004
    if epoch > total_epochs*0.8:
        lrate = 0.0008
    return lrate

'''
Placeholder loss
'''
def useless_loss(y_true,y_pred):
    zer = K.zeros(1)
    
    return zer
    
'''
Function to try to train the network on CIFAR10
'''
def main():
    epochs = 130
    batch_size = 64
    x_train,y_train,x_test,y_test = getCIFAR10()
    
    input_shape = getCIFAR10InputShape()
    print('CIFAR10 shape: ' + str(input_shape))
    
    model,m1,m2,m3 = getNetwork(input_shape)
    
    # define optimizer
    optim_sgd = optimizers.SGD(lr=0.1, decay=5e-4, momentum=0.9)
    
    model.compile(loss="categorical_crossentropy", optimizer=optim_sgd, metrics=["acc"])
    
    m1.compile(loss=[useless_loss,useless_loss],optimizer='sgd')
    m2.compile(loss=[useless_loss,useless_loss],optimizer='sgd')
    m3.compile(loss=[useless_loss,useless_loss],optimizer='sgd')
    
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.15,
        height_shift_range=0.15,
        horizontal_flip=True,  # randomly flip images
    )

    datagen.fit(x_train)

    model.fit_generator(
        datagen.flow(x_train, y_train, batch_size=batch_size),
        epochs=epochs,
        validation_data=(x_test, y_test),
        workers=4,
        shuffle=True,
        callbacks=[LearningRateScheduler(lr_schedule)])
    
    model_predictions = model.predict(x_train[0:1])
    m1_predictions = m1.predict(x_train[0:1])
    m2_predictions = m2.predict(x_train[0:1])
    m3_predictions = m3.predict(x_train[0:1])
    
    print('Full model predictions:')
    print(str(model_predictions))
    print("Model 1 predictions:")
    print(str(m1_predictions))
    print("Model 2 predictions:")
    print(str(m2_predictions))
    print("Model 3 predictions:")
    print(str(m3_predictions))
    
    saveModel(model,'./pretrained_models/wrn_16_2')
    saveModel(m1,'./pretrained_models/wrn_16_2_layer1')
    saveModel(m2,'./pretrained_models/wrn_16_2_layer2')
    saveModel(m3,'./pretrained_models/wrn_16_2_layer3')
    
main()



Using TensorFlow backend.


CIFAR10 loaded
CIFAR10 shape: (32, 32, 3)


/home/test/advanceddeep/our_code/ourwrnet.py:39: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), padding="same", kernel_initializer="he_normal", use_bias=False, kernel_regularizer=<keras.reg...)`
  use_bias=False)(input)
/home/test/advanceddeep/our_code/ourwrnet.py:51: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", strides=(1, 1), kernel_initializer="he_normal", use_bias=False, kernel_regularizer=<keras.reg...)`
  use_bias=False)(init)
/home/test/advanceddeep/our_code/ourwrnet.py:60: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", kernel_initializer="he_normal", use_bias=False, kernel_regularizer=<keras.reg...)`
  use_bias=False)(x)
/home/test/advanceddeep/our_code/ourwrnet.py:64: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (1, 1), padding="same", strides=(1, 1), kernel_initializer="he_normal", use_bias=False, kernel_regularizer=<keras

Wide Residual Network-16-2 created.


W1011 11:12:24.193294 139876948231552 deprecation_wrapper.py:119] From /home/test/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/130
782/782 [==============================] - 50s 64ms/step - loss: 2.3035 - acc: 0.3873 - val_loss: 1.7251 - val_acc: 0.5155
Epoch 2/130
782/782 [==============================] - 47s 60ms/step - loss: 1.5332 - acc: 0.5671 - val_loss: 1.4966 - val_acc: 0.5849
Epoch 3/130
782/782 [==============================] - 47s 61ms/step - loss: 1.2835 - acc: 0.6315 - val_loss: 1.6026 - val_acc: 0.5976
Epoch 4/130
782/782 [==============================] - 47s 60ms/step - loss: 1.1430 - acc: 0.6758 - val_loss: 1.3580 - val_acc: 0.6575
Epoch 5/130
782/782 [==============================] - 47s 60ms/step - loss: 1.0496 - acc: 0.7089 - val_loss: 1.2072 - val_acc: 0.6898
Epoch 6/130
782/782 [==============================] - 47s 61ms/step - loss: 0.9925 - acc: 0.7321 - val_loss: 1.0431 - val_acc: 0.7219
Epoch 7/130
782/782 [==============================] - 47s 60ms/step - loss: 0.9500 - acc: 0.7455 - val_loss: 1.0337 - val_acc: 0.7348
Epoch 8/130
782/782 [==============================] - 